<a href="https://colab.research.google.com/github/HK-MBURU/machine-learning-lessons/blob/master/Rule_Based_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m spacy download en_core_web_md

2024-01-16 13:55:07.646308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 13:55:07.646389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 13:55:07.647786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 13:55:08.921045: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import spacy
from spacy.matcher import Matcher

nlp=spacy.load("en_core_web_md")
matcher=Matcher(nlp.vocab)

pattern=[{"LOWER":"feb"},{"IS_DIGIT":True},{"IS_PUNCT":True},{"IS_DIGIT":True}]
matcher.add("DATE",[pattern])
doc=nlp("Your expected delivery date is feb 8, 2021")

matches=matcher(doc)
for match_id,start,end in matches:
  print(doc[start:end])

feb 8, 2021


**Regex Matcher**

In [ ]:
import spacy
import re

nlp= spacy.load("en_core_web_md")
doc=nlp("The United States of Americ (USA) are comonly known as the United States (U.S. or US) or America.")

expression=r"U[.]?S[.]?"
for match in re.finditer(expression,doc.text):
  start,end=match.span()
  span=doc.char_span(start,end)
  if span is not None:
    print("Found match:", span.text)

Found match: U.S.
Found match: US


# Phrase **Matcher**

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
nlp=spacy.load("en_core_web_md")
matcher=PhraseMatcher(nlp.vocab)

# create the list of words to match
lang_list=['Python','C++','Java']

# obtain doc object for each word in the list and store it in a list
patterns=[nlp(lang) for lang in lang_list]

# add the pattern to the matcher
matcher.add("PROGRAMMING_LANGUAGE",patterns)

# Process some text
doc=nlp("Python requires less typing, provides new libraries, fast prototyping and several other new features. C++ as of today in its ")
matches=matcher(doc)
for match_id, start, end in matches:
  span=doc[start:end]
  print(span.text)

Python
C++


# **Extract information from invoice**

In [ ]:
!apt install build-essential libpoppler-cpp-dev pkg-config python3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 50.4 kB of archives.
After this operation, 231 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-cpp0v5 amd64 22.02.0-2ubuntu0.3 [38.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-cpp-dev amd64 22.02.0-2ubuntu0.3 [11.7 kB]
Fetched 50.4 kB in 0s (129 kB/s)
Selecting previously unselected package libpoppler-cpp0v5:amd64.
(Reading database ... 12165

In [ ]:
!pip install pdftotext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pdftotext: filename=pdftotext-2.2.2-cp310-cp310-linux_x86_64.whl size=61850 sha256=d5a34bbded9cfd82f822e8105a6cc69bd31a07910ae67c91724f9ecb5b873c7c
  Stored in directory: /root/.cache/pip/wheels/c8/68/c6/82a8c2046e1a8f9051017319dc007f43ef3a6edc53149d1f4e
Successfully built pdftotext


In [ ]:
import pdftotext

# load your pdf
with open("/content/invoice-test.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

In [ ]:
for page in pdf:
    print(page)

In [ ]:
import re

In [ ]:
amount_due=r"Amount[ ]Due[ ]USD[ ]*$\d*[ .,\d]*\d"
re.findall(amount_due,"Amount Due (USD)                    $1,837.83")

In [ ]:
date=r"(0[0-9]|1[0-9]|2[0-9]|3[0-1])[/](0[0-9]|1[0-2])[/]([1-2][0-9][0-9][0-9])"
re.findall(date," 06/09/2021  07/09/2021")

In [ ]:
invoice_number="[0-9]{7}"
re.findall(invoice_number,"hello      1234567     6575812      ")

In [ ]:
import re
import spacy
from spacy.tokens import Span
from spacy.lang.en import English
from spacy.language import Language


@Language.component('regex_matcher')
def regex_matcher(doc):
  expressions={
      "amount_due": re.compile(r"Amount[ ]Due[ ]USD[ ]*$\d*[ .,\d]*\d"),
      "date": re.compile(r"(0[0-9]|1[0-9]|2[0-9]|3[0-1])[/](0[0-9]|1[0-2])[/]([1-2][0-9][0-9][0-9])"),
      "invoice_number": re.compile(r"[0-9]{7}")
  }
  spans=[]
  for label, expression in expressions.items():
      for match in re.finditer(expression, doc.text):
          start, end = match.span()
          entity = doc.char_span(start, end, label=label)
          if entity:
            spans.append(entity)

  doc.ents=list(doc.ents)+spacy.util.filter_spans(spans)
  return doc

In [ ]:
nlp = spacy.load('en_core_web_md', disable = ['ner'])
nlp.add_pipe('regex_matcher')
doc = nlp(pdf[0])

for ent in doc.ents:
    print(ent, ent.label_)


In [ ]:
from spacy import displacy
displacy.render(doc, style='ent',jupyter=True)